# atmodeller

## Tutorial 1: basic operation

Import the required functionality and activate the debug logger.

In [ ]:
from atmodeller import debug_logger #, debug_file_logger
from atmodeller.constraints import MassConstraint, SystemConstraints, IronWustiteBufferConstraintHirschmann, FugacityConstraint
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Species
from atmodeller.interfaces import ThermodynamicDataJANAF, GasSpecies
from atmodeller.solubilities import BasaltDixonCO2, PeridotiteH2O
from atmodeller.utilities import earth_oceans_to_kg
import logging

# debug_file_logger()
logger = debug_logger()
logger.setLevel(logging.INFO)

### 1. Simple H2O-H2-O2 system

We define a list of the species we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere via a solubility law:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))


We can then create an interior-atmosphere system using the list of species and (optionally) specifying the thermodynamic data to use. Note that this creates a planet with 'default properties' (a molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [ ]:
interior_atmosphere = InteriorAtmosphereSystem(species=species)

To solve the system, we provide a constraint of the H2O and O2 fugacity in bar. By default, the O2 fugacity is constrained at the IW buffer:

In [ ]:
H2O_fugacity  = FugacityConstraint(species='H2O', value=1)
O2_fugacity = IronWustiteBufferConstraintHirschmann()
constraints = SystemConstraints([H2O_fugacity, O2_fugacity])

In [ ]:
interior_atmosphere.solve(constraints)

You can access the solution directly using:

In [ ]:
interior_atmosphere.solution_dict

You can access more information about the solution using:

In [ ]:
interior_atmosphere.output

### 2. System with C and H and prescribed pressures

We now extend the species list to additionally include C-species:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
# species.append(GasSpecies(chemical_formula='O2S', solubility=BasaltS()))
# species.append(GasSpecies(chemical_formula='OS', solubility=BasaltS()))
# species.append(GasSpecies(chemical_formula='S2', solubility=BasaltS()))

species

In [ ]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [ ]:
H2O_fugacity = FugacityConstraint(species='H2O', value=1)
CO2_fugacity = FugacityConstraint(species='CO2', value=1)
O2_fugacity = IronWustiteBufferConstraintHirschmann(log10_shift=3)
constraints = SystemConstraints([H2O_fugacity, CO2_fugacity, O2_fugacity])

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three fugacity constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution.

In [ ]:
H2O_fugacity = FugacityConstraint(species='H2O', value=1)
H2_fugacity = FugacityConstraint(species='H2', value=1)
CO_fugacity = FugacityConstraint(species='CO', value=1)
constraints = SystemConstraints([H2O_fugacity, H2_fugacity, CO_fugacity])

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of fugacity and mass constraints. We define the same species as before:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [ ]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans)
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    IronWustiteBufferConstraintHirschmann()
])

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)
interior_atmosphere.solve(constraints, factor=1)
interior_atmosphere.solution_dict

### 4. Including more species such as CH4

It is straightforward to add more species to the system:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='CH4'))
species

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [ ]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans)
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    IronWustiteBufferConstraintHirschmann()
])

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict